In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install triton
!pip install xformers
!pip install bitsandbytes

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.9 MB/s eta 0:00:00


In [1]:
!pip install unsloth

In [2]:
!apt install nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java libatk-wrapper-java-jni
  libbabeltrace1 libcub-dev libcublas11 libcublaslt11 libcudart11.0 libcufft10 libcufftw10
  libcuinj64-11.5 libcupti-dev libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libfontenc1 libgail-common libgail18
  libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev libglx-dev
  libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11 libnppidei11
  libnppif11 libnppig11 libnppim11 libnppist11 libnppisu11 libnppitc11 libnpps11 libnvblas11
  libnvidia-compute-495 libnvidia-compute-510 libnvidia-compute-535 libnvidia-ml-dev libnvjpeg11
  libnvrtc-builtins11.5 libnvrtc11.2 libnvtoolsext1 libnvvm4 libopengl-dev libr

In [4]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 8192
model, tokenizer = FastLanguageModel.from_pretrained(
     model_name = "elyza/Llama-3-ELYZA-JP-8B" ,
     max_seq_length = max_seq_length,
     dtype = None ,
     load_in_4bit = True ,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.10.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    mapping = {"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    map_eos_token = True
)

Unsloth: Will map <|im_end|> to EOS = <|eot_id|>.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def formatting_prompts_func(examples):
    # Removed the recursive call to load_dataset and map
    convos = examples["conversations"]
    texts = [f'<bos>{tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False)}' for convo in convos]
    return { "text" : texts, }

dataset = load_dataset("nappa0326/glaive-function-calling-v2-sharegpt-japanese", split = "train")
# The formatting_prompts_func is now only called once in the map function
dataset = dataset.map(formatting_prompts_func, batched = True,)

print(dataset[5]["text"])

README.md:   0%|          | 0.00/2.51k [00:00<?, ?B/s]

(…)-00000-of-00002-6f3344faa23e9b0a.parquet:   0%|          | 0.00/98.0M [00:00<?, ?B/s]

(…)-00001-of-00002-41f063cddf49c933.parquet:   0%|          | 0.00/98.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112960 [00:00<?, ? examples/s]

Map:   0%|          | 0/112960 [00:00<?, ? examples/s]

<bos><|im_start|>system
You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "search_movies",
    "description": "Search for movies based on title or genre",
    "parameters": {
        "type": "object",
        "properties": {
            "search_query": {
                "type": "string",
                "description": "The title or genre of the movie"
            }
        },
        "required": [
            "search_query"
        ]
    }
}
<|im_end|>
<|im_start|>user
Can you find me some horror movies?<|im_end|>
<|im_start|>assistant
<functioncall> {"name": "search_movies", "arguments": '{"search_query": "horror"}'} <|endoftext|><|im_end|>
<|im_start|>system
{"movies": ["The Conjuring", "Insidious", "A Quiet Place", "Hereditary", "Get Out"]}<|im_end|>
<|im_start|>assistant
Here are some horror movies you might like: "The Conjuring", "Insidious", "A Quiet Place", "Hereditary", "Get Out". <|endoftext|><|im_end|>
<|im_start|>user
Wh

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    tokenizer = tokenizer,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 1000,
        save_strategy = "steps",
        save_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        output_dir = "llama3-elyza-jp-ft-function-calling",
        # num_train_epochs = 1,
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
    ),
)

Map (num_proc=2):   0%|          | 0/112960 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# Check if the output directory exists and create it if it doesn't
import os
if not os.path.exists("llama3-elyza-jp-ft-function-calling"):
    os.makedirs("llama3-elyza-jp-ft-function-calling")

# Modify the training loop to handle the error
try:
    trainer_stats = trainer.train(resume_from_checkpoint="llama3-elyza-jp-ft-function-calling")
except ValueError as e:
    if "Can't find a valid checkpoint" in str(e):
        print("No valid checkpoint found. Starting training from scratch.")
        trainer_stats = trainer.train(resume_from_checkpoint=False)
    else:
        raise e  # Re-raise the exception if it's not the specific error we're handling

No valid checkpoint found. Starting training from scratch.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 112,960 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.046300
2,2.283700
3,2.062000
4,1.415200
5,1.357300
6,1.283900
7,1.334400
8,0.999300
9,0.947800
10,0.751300


In [ ]:
model.save_pretrained_gguf("gguf", tokenizer, quantization_method = "q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 16.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.7 out of 12.67 RAM for saving.


100%|██████████| 32/32 [02:26<00:00,  4.58s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving gguf/pytorch_model-00001-of-00004.bin...
Unsloth: Saving gguf/pytorch_model-00002-of-00004.bin...
Unsloth: Saving gguf/pytorch_model-00003-of-00004.bin...
Unsloth: Saving gguf/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at gguf into f16 GGUF format.
The output location will be /content/gguf/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_q.weight,         torch.float1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.push_to_hub_gguf("nappa0326/llama-3-elyza-jp-8b-ft-functioncalling-gguf", tokenizer, quantization_method = "q4_k_m", token = "")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.87 out of 12.67 RAM for saving.


100%|██████████| 32/32 [02:26<00:00,  4.58s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving nappa0326/llama-3-elyza-jp-8b-ft-functioncalling-gguf/pytorch_model-00001-of-00004.bin...
Unsloth: Saving nappa0326/llama-3-elyza-jp-8b-ft-functioncalling-gguf/pytorch_model-00002-of-00004.bin...
Unsloth: Saving nappa0326/llama-3-elyza-jp-8b-ft-functioncalling-gguf/pytorch_model-00003-of-00004.bin...
Unsloth: Saving nappa0326/llama-3-elyza-jp-8b-ft-functioncalling-gguf/pytorch_model-00004-of-00004.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at nappa0326/llama-3-elyza-jp-8b

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.F16.gguf:   0%|          | 0.00/16.1G [00:00<?, ?B/s]

...

unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/nappa0326/llama-3-elyza-jp-8b-ft-functioncalling-gguf


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


Saved Ollama Modelfile to https://huggingface.co/nappa0326/llama-3-elyza-jp-8b-ft-functioncalling-gguf
